<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/FileLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Hemodialysis.csv', encoding='latin1')

In [3]:
lab = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Lab.csv')

In [4]:
slope = pd.read_excel('/content/drive/MyDrive/Thesis Project Kidney/Assign/Assign 3: Re-CountSlope & Quartile/LabSlope.xlsx')

In [5]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()
# Remove patient UIDs from lab['PatientUID'] that are not in the list
lab = lab[~lab['PatientUID'].isin(patient_uid_list)]
lab.shape

(17728574, 11)

In [6]:
lab_location = lab['Location'].value_counts().reset_index()
lab_location.columns = ['Location', 'Location_Count']

lab_result = lab['ResultItemName'].value_counts().reset_index()
lab_result.columns = ['ResultItemName', 'ResultItemName_Count']

lab_request = lab['RequestItemName'].value_counts().reset_index()
lab_request.columns = ['RequestItemName', 'RequestItemName_Count']

columns_to_count = ['Location', 'ResultItemName', 'RequestItemName']

# Count unique values in the specified columns
lab_count = lab[columns_to_count].value_counts().reset_index()
lab_count.columns = ['Location', 'ResultItemName', 'RequestItemName', 'Count']

lab_df = pd.DataFrame(lab_count)

In [7]:
#lab[lab['ResultValue'].isnull()] #196

# **Slope**


> **ค่า x = ค่าจากวิสิท t ที่มาก่อน**




In [25]:
data = pd.DataFrame(slope)
data = data[['PatientUID','PatientVisitUID','Slopes']]
data = data.dropna()

In [26]:
df = pd.DataFrame(data)

# Sort the DataFrame by PatientUID and PatientVisitUID
df = df.sort_values(by=['PatientUID', 'PatientVisitUID'])

# Create a new column to store the previous slope value
df['PreviousSlope'] = df['Slopes'].shift(1)

# Update the Slopes column with the PreviousSlope values
df['Slopes'] = df['PreviousSlope']

# Drop the PreviousSlope column
df = df.drop(columns=['PreviousSlope'])
data = df.dropna()

In [30]:
LabSummaryDF.columns

Index(['Location', 'ResultItemName', 'RequestItemName', 'Count'], dtype='object')

In [11]:
LabSummaryDF = pd.DataFrame(lab_df)

In [12]:
LabSummaryDF.columns

Index(['Location', 'ResultItemName', 'RequestItemName', 'Count'], dtype='object')

In [13]:
lab.columns

Index(['LabNo', 'PatientUID', 'PatientVisitUID', 'Location', 'RequestItemName',
       'ResultItemName', 'CWhen', 'ResultDttm', 'ResultValue', 'Unit',
       'ReferenceRange'],
      dtype='object')

# **List of LabValue**

In [14]:
# Filter lab data based on indices in LabSummaryDF
lab1 = LabSummaryDF[LabSummaryDF.index.isin(lab.index[[3, 129, 185, 312]])] # Using indices 1 and 3 from LabSummaryDF

# Create the BUN list containing ResultItemName values
BUN = lab1['ResultItemName'].tolist()

# Use .loc to update values in the ResultItemName column
lab.loc[lab['ResultItemName'].isin(BUN), 'ResultItemName'] = 'BUN'

In [ ]:
import pandas as pd

# Assuming you have the 'data', 'lab', and 'LabSummaryDF' DataFrames

lab1 = []  # Initialize the list to store lab values

for pid, vid in zip(data['PatientUID'], data['PatientVisitUID']):
    # Filter labDF for the current patient ID and visit ID
    subsetLabDF = lab[(lab['PatientUID'] == pid) & (lab['PatientVisitUID'] == vid)]

    pVLabList = subsetLabDF['ResultItemName']  # Get the list of lab items

    for labValue in LabSummaryDF.loc[[0, 103], 'ResultItemName']:
        if labValue in pVLabList.values:
            # Get the corresponding ResultValue
            labResultValue = subsetLabDF.loc[subsetLabDF['ResultItemName'] == labValue, 'ResultValue'].values[0]
            lab1.append(labResultValue)
        else:
            lab1.append(None)  # Append None if labValue is not found

# Add the lab1 list as a new column in the 'data' DataFrame
data['lab1'] = lab1
